In [1]:
from transformers import pipeline

/home/js/anaconda3/envs/ai/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/js/anaconda3/envs/ai/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/js/anaconda3/envs/ai/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
classifier(
    [
        "I've been waiting for a course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9450951814651489},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [4]:
from transformers import AutoTokenizer

In [5]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
raw_inputs = [
    "I've been waiting for a course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 2607, 2026, 2878, 2166,
         1012,  102],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102,    0,    0,    0,    0,
            0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}


In [8]:
from transformers import AutoModel

In [10]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [16]:
output = model(**inputs)
print(output.last_hidden_state)
print(output.last_hidden_state.shape)

tensor([[[ 0.0265,  0.4913,  0.1807,  ..., -0.2614,  0.1424,  0.3351],
         [ 0.4510,  1.0154, -0.0511,  ..., -0.2005,  0.3131,  0.3615],
         [ 1.0264,  0.1236,  0.1441,  ...,  0.2194, -0.1873, -0.5749],
         ...,
         [ 0.5826,  0.7445, -0.0627,  ..., -0.6752,  0.4747,  0.1195],
         [ 0.6823,  0.1428, -0.0272,  ...,  0.2854, -0.0647, -0.5627],
         [ 0.3957,  0.6452,  0.1107,  ...,  0.2246,  0.2926, -0.1576]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.2201,  0.8474, -0.0195,  ..., -0.0863, -0.8485, -0.1036],
         [-0.2319,  0.8268, -0.0312,  ..., -0.0764, -0.8509, -0.1043],
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861]]],
       grad_fn=<NativeLayerNormBackward0>)
torch.Size([2, 14, 768])


In [17]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)


In [19]:
print(outputs.logits)
print(outputs.logits.shape)

tensor([[-1.4078,  1.4378],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
torch.Size([2, 2])


In [20]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[5.4905e-02, 9.4509e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [21]:
model.config.id2label


{0: 'NEGATIVE', 1: 'POSITIVE'}

In [ ]:
#2

In [22]:
from transformers import BertConfig, BertModel

In [23]:
config = BertConfig()
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


In [24]:
model = BertModel(config)

In [25]:
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [27]:
model.save_pretrained("saving_model")

In [28]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [29]:
import torch

In [30]:
model_inputs = torch.tensor(encoded_sequences)
print(model_inputs)

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])


In [31]:
output = model(model_inputs)
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9394e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6738e-01, -1.8187e-01,  ...,  2.4671e-01,
           1.0441e+00, -6.1970e-03]],

        [[ 3.6436e-01,  3.2464e-02,  2.0258e-01,  ...,  6.0111e-02,
           3.2451e-01, -2.0995e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4877e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1079e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1320e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2

In [32]:
tokenized_text = "hi my name is jongsoo lee".split()
print(tokenized_text)

['hi', 'my', 'name', 'is', 'jongsoo', 'lee']


In [33]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [35]:
tokenizer("i am your father")

{'input_ids': [101, 178, 1821, 1240, 1401, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [36]:
tokenizer.save_pretrained("saving_tokenizer")

('saving_tokenizer/tokenizer_config.json',
 'saving_tokenizer/special_tokens_map.json',
 'saving_tokenizer/vocab.txt',
 'saving_tokenizer/added_tokens.json',
 'saving_tokenizer/tokenizer.json')

In [37]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [39]:
sequences = "using a transformer network is simple"

tokens = tokenizer.tokenize(sequences)
print(tokens)

['using', 'a', 'transform', '##er', 'network', 'is', 'simple']


In [40]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1606, 170, 11303, 1200, 2443, 1110, 3014]


In [41]:
output = tokenizer(sequences)
print(output)

{'input_ids': [101, 1606, 170, 11303, 1200, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [48]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

using a transformer network is simple


In [52]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = "using a transformer network is simple"

tokens = tokenizer.tokenize(sequences)
print(tokens)

['using', 'a', 'transform', '##er', 'network', 'is', 'simple']


In [53]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2478, 1037, 10938, 2121, 2897, 2003, 3722]


In [ ]:
## 4 다중 시퀀스 처리

In [49]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [50]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [54]:
sequences = "I've been waiting for a huggingFace course my whole life."

In [64]:
tokens = tokenizer.tokenize(sequences)
print("token:", tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print("ids:",ids)
input_ids = torch.tensor(ids)
print("input_ids:", input_ids)

token: ['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
ids: [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
input_ids: tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])


In [65]:
model(input_ids=input_ids)

RuntimeError: The size of tensor a (14) must match the size of tensor b (512) at non-singleton dimension 1

In [66]:
tokens = tokenizer.tokenize(sequences)
print("token:", tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print("ids:",ids)
input_ids = torch.tensor([ids])
print("input_ids:", input_ids)

token: ['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
ids: [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
input_ids: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


In [68]:
output = model(input_ids=input_ids)
print("logits:", output.logits)

Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [69]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [72]:
output = model(torch.tensor(model_inputs.input_ids))
print("logits:",output.logits)

ValueError: expected sequence of length 16 at dim 1 (got 6)

In [81]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

example = "My name is Wolfgang and I live in Berlin"

inputs = tokenizer(example, return_tensors="pt")
print(inputs)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  1422,  1271,  1110, 14326,  1105,   146,  1686,  1107,  3206,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [82]:
outputs = model(**inputs)
print(outputs)

TokenClassifierOutput(loss=None, logits=tensor([[[ 7.0687e+00, -4.5536e-01, -1.0750e+00, -8.5831e-01, -1.1361e+00,
          -9.6533e-01, -1.5936e+00, -9.7993e-01, -1.1887e+00],
         [ 9.3086e+00, -1.2812e+00, -2.5905e+00,  7.8382e-03, -1.9809e+00,
          -1.5162e+00, -2.3603e+00, -6.7013e-01, -1.2216e+00],
         [ 9.6154e+00, -1.1142e+00, -2.0568e+00, -3.5809e-01, -1.1853e+00,
          -2.4836e+00, -1.7056e+00, -1.2519e+00, -1.1425e+00],
         [ 9.5420e+00, -1.0611e+00, -2.0022e+00, -3.3472e-01, -1.2728e+00,
          -2.5949e+00, -1.7528e+00, -1.1393e+00, -9.9801e-01],
         [-4.8635e-01, -1.4142e+00, -2.8361e+00,  8.0971e+00, -5.2416e-01,
          -5.9288e-01, -2.8314e+00, -8.1996e-03, -2.0378e+00],
         [ 9.7367e+00, -1.5220e+00, -1.6149e+00, -1.0701e+00, -7.6580e-01,
          -2.2765e+00, -1.1857e+00, -1.3290e+00, -1.3701e+00],
         [ 9.2872e+00, -1.4188e+00, -2.6495e+00, -4.5131e-02, -1.6068e+00,
          -1.8935e+00, -2.1403e+00, -7.0498e-01, -1.2672e

In [84]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
print(model_inputs)

{'input_ids': [[101, 146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119, 102], [101, 1573, 1138, 146, 106, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [86]:
# 해당 시퀀스를 리스트 내의 최대 시퀀스 길이까지 패딩(padding) 합니다.
model_inputs = tokenizer(sequences, padding="longest")
print(model_inputs)

{'input_ids': [[101, 146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119, 102], [101, 1573, 1138, 146, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [87]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119, 102]
[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119]


In [88]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] I've been waiting for a HuggingFace course my whole life. [SEP]
I've been waiting for a HuggingFace course my whole life.


In [89]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
